# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split, ParameterSampler, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy import stats

# Utilizadas en el punto 2.2 para graficar las distintas configuraciones del modelo
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
#dataset = pd.read_csv("./data/loan_data.csv", comment="#")
file = "data/loan_data.csv"
dataset = pd.read_csv(file, comment="#")

In [ ]:
# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

https://www.kaggle.com/ajay1735/hmeq-data

1. El Home Equity dataset (HMEQ), contiene una linea base y la información de rendimiento de los préstamos de 5690 préstamos otorgados recientemente. Esta planilla tiene 1854 registros.

2. El variable objetivo (BAD) es una variable binaria que indica si un solicitante finalmente cumplió o incumplió en la cancelación del préstamo solicitado.  
 * 1 = cliente incumplió con el préstamo 
 * 0 = préstamo cancelado

3. Atributos:
 * LOAN    Amount of the loan request. Monto del préstamo solicitado.
 * MORTDUE Amount due on existing mortgage. Monto adeudado de la hipoteca.
 * VALUE   Value of current property. Valor actual de la propiedad.
 * YOJ     Years at present job. Antiguedad en el trabajo actual.
 * DEROG   Number of major derogatory reports. Número de informes negativos importantes.
 * DELINQ  Number of delinquent credit lines. Cantidad de créditos no pagados.
 * CLAGE   Age of oldest trade line in months. Edad de la línea comercial más antigua en meses.
 * NINQ    Number of recent credit lines. Número de líneas de crédito recientes.
 * CLNO    Number of credit lines. Número de linas de crédito.
 * DEBTINC Debt-to-income ratio. Relación Deuda / Ingresos.

4. Seleccionaría:
 * DEROG   Number of major derogatory reports. Número de informes negativos importantes.
 * DELIQ   Number of delinquent credit lines. Cantidad de créditos no pagados.
 * CLNO    Number of credit lines. Número de linas de crédito.
 * DEBTINC Debt-to-income ratio. Relación Deuda / Ingresos.




In [ ]:
dataset.describe()

In [ ]:
dataset[dataset["DEROG"] == 10]

In [ ]:
dataset['TARGET'].value_counts().plot.bar();

In [ ]:
features = dataset.columns[1:]
fig = plt.figure(figsize=(15,5*len(features)))
ds0 = dataset[dataset.TARGET == 0]
ds1 = dataset[dataset.TARGET == 1]
with sns.axes_style("white"):
    for i, col in enumerate(features):
        sp1 = plt.subplot(len(features), 2, i+1)
        sns.distplot(ds0[col], label='0')
        sns.distplot(ds1[col], label='1')
        plt.legend()

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


SGDClassifier es sensitivo a las magnitudes de los valores, por esta razón a continuación se realiza un escalado o normalización.

In [ ]:
escalado = StandardScaler()
escalado.fit(X_train)
X_train = escalado.transform(X_train)
X_test  = escalado.transform(X_test) 

### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
model = SGDClassifier(random_state=0)
classifier=model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

Tomando como True = 1:

$$Accuracy = \frac{TP + TN}{TP + FP + FN + TN} = \frac{105 + 1190}{105 + 42 + 146 + 1190} = 0.87 $$

$$Precision = \frac{TP}{TP + FP} = \frac{105}{105 + 42} = 0.71 $$

$$Recall = \frac{TP}{TP + FN} = \frac{105}{105 + 146} = 0.42 $$

$$F_1 = \frac{2 * Precision * Recall}{Precision + Recall} = \frac{2 * 0.71 * 0.42}{0.71 + 0.42} = 0.53 $$


In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
print('TP: ', tp, '\tFP: ', fp, '\nTN: ', tn, '\tFN: ', fn)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(14,12))
fig.suptitle('Matrices de confusión SGDClassifier por defecto')
axs[0][0].title.set_text('Entrenamiento')
axs[0][1].title.set_text('Evaluación')
fig.tight_layout(pad=5.0)
np.set_printoptions(precision=2)
titles_options = [("Entrenamiento", 0, 0, X_train, y_train, None, 'd'),
                  ("Evaluación", 0, 1, X_test, y_test,  None, 'd'),
                  ("Entrenamiento normalizada", 1, 0, X_train, y_train, 'true', None),
                  ("Evaluación normalizada", 1, 1, X_test, y_test, 'true', None)
                  ]
for title, row, col, x_val, y_val, norm, val_format in titles_options:
    disp = plot_confusion_matrix(classifier, x_val, y_val,
#                                  display_labels=class_names,
                                 cmap=plt.cm.Greens_r,
                                 normalize=norm,
                                 values_format=val_format,
                                 ax=axs[row][col]
                                 )
    disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
param_grid = {
    'loss': [
        'hinge',        # SVM
        'log',          # logistic regression
    ],
#     'learning_rate': [
#         'constant',
#         'optimal',
#         'invscaling', 
#         'adaptive', 
#     ],
#     'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0], 
#     'eta0': [0.01, 0.1, 1] 
    'learning_rate': ['optimal','constant', 'adaptive'],  #diferents opciones de tasa de entrenamiento
#     'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'], #diferentes opciones de función de costo
    'l1_ratio': list(np.linspace(0, 1, 4))+[0.15], # diferentes opciones de tasas de regularización
    'alpha': np.logspace(-4, 0, 4),
    'eta0': np.logspace(-2, 0, 2),
    
}

In [ ]:
#cv=None is cv=5 by default.. cross validation 
model = SGDClassifier(random_state=0, max_iter=3000)
cv = GridSearchCV(model, param_grid, scoring='accuracy')
cv.fit(X_train, y_train); #entrenamiento o dataset completo?
#cv.fit(X, y) 

In [ ]:
# SGDClassifier()

In [ ]:
results = cv.cv_results_
df = pd.DataFrame(results)

In [ ]:
df.describe()

In [ ]:
pd.options.display.max_colwidth = 100
df[['params','mean_test_score','std_test_score','rank_test_score']].head(20)

In [ ]:
df.sort_values(['mean_test_score','std_test_score','rank_test_score'],
               ascending = [False,True,True],
               inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
fig = go.Figure()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df.index, y=df['mean_test_score'], 
                         name='Mean score', mode='markers', marker=dict(color='royalblue'),
                         yaxis='y1',
                         text=df['params'].astype('str')
                         )
             )

fig.add_trace(go.Scatter(x=df.index, y=df['std_test_score'], 
                         name='Std score', mode='markers', marker=dict(color='green'),
                         yaxis = 'y2',
                         text=df['params'].astype('str')
                         )
             )

fig.update_layout(
    title=go.layout.Title(
        text="Reporte para {} configuraciones de parámetros".format(len(df)),
        xref="paper",
        x=0
    ),
    xaxis_title="ranking",
    yaxis_title="Mean test score",
    yaxis2_title="Std test score", 
 )


Seleccionamos la mejor configuración de hyper parámetros.

In [ ]:
best_model = df.loc[df['rank_test_score'].idxmin()]['params']
best_model

In [ ]:
model = SGDClassifier(**best_model, random_state=0)
classifier = model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(14,12))
fig.suptitle('Matrices de confusión SGDClassifier mejor modelo')
axs[0][0].title.set_text('Entrenamiento')
axs[0][1].title.set_text('Evaluación')
fig.tight_layout(pad=5.0)
np.set_printoptions(precision=2)
titles_options = [("Entrenamiento", 0, 0, X_train, y_train, None, 'd'),
                  ("Evaluación", 0, 1, X_test, y_test,  None, 'd'),
                  ("Entrenamiento normalizada", 1, 0, X_train, y_train, 'true', None),
                  ("Evaluación normalizada", 1, 1, X_test, y_test, 'true', None)
                  ]
for title, row, col, x_val, y_val, norm, val_format in titles_options:
    disp = plot_confusion_matrix(classifier, x_val, y_val,
#                                  display_labels=class_names,
                                 cmap=plt.cm.Greens_r,
                                 normalize=norm,
                                 values_format=val_format,
                                 ax=axs[row][col]
                                 )
    disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [ ]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print('Profundidad máxima del árbol:', model.get_depth())
print('Cantidad máxima de hojas:', model.get_n_leaves())

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

Graficamos el árbol de decisión.

In [ ]:
plt.figure(figsize=(17,10))
plot_tree(model, impurity=False, fontsize=8, filled=True, label='root')  #filled=True,
plt.show()

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
param_grid = {
    'splitter': ['best', 'random'],
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(4,20,2),
    'min_samples_leaf': np.arange(1,10),
}


In [ ]:
cv = GridSearchCV(DecisionTreeClassifier(random_state=0), param_grid, scoring='accuracy')
cv.fit(X, y);

In [ ]:
results = cv.cv_results_
df = pd.DataFrame(results)

In [ ]:
df.sort_values(['mean_test_score','std_test_score','rank_test_score'],
               ascending = [False,True,True],
               inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
fig = go.Figure()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df.index, y=df['mean_test_score'], 
                         name='Mean score', mode='markers', marker=dict(color='royalblue'),
                         yaxis='y1',
                         text=df['params'].astype('str')
                         )
             )

fig.add_trace(go.Scatter(x=df.index, y=df['std_test_score'], 
                         name='Std score', mode='markers', marker=dict(color='green'),
                         yaxis = 'y2',
                         text=df['params'].astype('str')
                         )
             )

fig.update_layout(
    title=go.layout.Title(
        text="Reporte para {} configuraciones de parámetros".format(len(df)),
        xref="paper",
        x=0
    ),
    xaxis_title="ranking",
    yaxis_title="Mean test score",
    yaxis2_title="Std test score", 
 )


Seleccionamos la mejor configuración de hyper parámetros.

In [ ]:
best_model = df.loc[df['rank_test_score'].idxmin()]['params']
best_model

In [ ]:
model = DecisionTreeClassifier(**best_model, random_state=0)
classifier=model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print('Profundidad máxima del árbol:', model.get_depth())
print('Cantidad máxima de hojas:', model.get_n_leaves())

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
print('TP: ', tp, '\t FP: ', fp, '\nTN: ', tn, '\t FN: ', fn)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(14,12))
fig.suptitle('Matrices de confusión DecisionTreeClassifier mejor modelo')
axs[0][0].title.set_text('Entrenamiento')
axs[0][1].title.set_text('Evaluación')
fig.tight_layout(pad=5.0)
np.set_printoptions(precision=2)
titles_options = [("Entrenamiento", 0, 0, X_train, y_train, None, 'd'),
                  ("Evaluación", 0, 1, X_test, y_test,  None, 'd'),
                  ("Entrenamiento normalizada", 1, 0, X_train, y_train, 'true', None),
                  ("Evaluación normalizada", 1, 1, X_test, y_test, 'true', None)
                  ]
for title, row, col, x_val, y_val, norm, val_format in titles_options:
    disp = plot_confusion_matrix(classifier, x_val, y_val,
#                                  display_labels=class_names,
                                 cmap=plt.cm.Greens_r,
                                 normalize=norm,
                                 values_format=val_format,
                                 ax=axs[row][col]
                                 )
    disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

Graficamos el árbol de decisión.

In [ ]:
plt.figure(figsize=(17,10))
plot_tree(model, impurity=False, fontsize=8, filled=True, label='root')  #filled=True,
plt.show()

## Conclusión

Observamos que el modelo de árbol de decisión fue más preciso que el modelo descenso de graciente estocástico.

También observamos que los datos están un poco desbalanceados. De los 1854 préstamos, sólo un 25% corresponden a préstamos no cancelados. 
